In [1]:
import pandas as pd 
import sys
import random

In [2]:
def get_data_file(bencana):
    d_anotated = f'D:\\data-gusman\\tmp\\anotated_data\\{bencana}\\v02-new\\'
    return f_data

In [3]:
f_data = get_data_file('erupsi')
%run analysis_repr.py $f_data

Selection of representation - text
Text selection: title only vs content only vs title and content
Algorithm:  Multinomial NB
Feature:   BOW - Unigram
      t_lower_acc  c_lower_acc  tc_lower_acc  t_lower_P  c_lower_P  \
0        0.880734     0.848624      0.848624   0.531250   0.454545   
1        0.853211     0.825688      0.825688   0.380952   0.366667   
2        0.885321     0.844037      0.848624   0.571429   0.475410   
3        0.894495     0.885321      0.880734   0.515152   0.488889   
4        0.885321     0.834862      0.839450   0.729730   0.541667   
MEAN     0.879817     0.847706      0.848624   0.545702   0.465436   

      tc_lower_P  t_lower_R  c_lower_R  tc_lower_R  t_lower_F1  c_lower_F1  \
0       0.454545   0.607143   0.892857    0.892857    0.566667    0.602410   
1       0.366667   0.727273   1.000000    1.000000    0.500000    0.536585   
2       0.483333   0.774194   0.935484    0.935484    0.657534    0.630435   
3       0.478261   0.708333   0.916667    0.91